In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('../input/Amazon_Unlocked_Mobile.csv')
df = df.sample(frac=0.1, random_state=10)

In [2]:
print(df.shape)
df.head()

(41384, 6)


,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
394349,Sony XPERIA Z2 D6503 FACTORY UNLOCKED Internat...,NaN,244.95,5,Very good one! Better than Samsung S and iphon...,0.0
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0


In [3]:
df.dropna(inplace=True)
df.shape

(33408, 6)

In [4]:
def f(x):
    if x['Rating']==3 : return 'neutral'
    elif x['Rating']>3 : return 'positive'
    else : return 'negative'
    
df['Positive Review'] = df.apply(f,axis=1)

In [5]:
df['Positive Review'] = np.where(df['Rating']==3,'neutral',np.where(df['Rating']>3,'positive','negative'))

In [6]:
x = df['Reviews']
y = df['Positive Review']
df[df['Rating']==3]['Reviews'].iloc[1]


"I bought this for a travel phone. It was way too small for easy use. If you don't want to type anything then it would be fine. But, I am used to a smartphone and this just didn't work for me. I bought an Acer phone in the Munich airport and used that for my trip."

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [8]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.3,stratify = y)

In [9]:
v =  CountVectorizer(ngram_range = [1,2])
v.fit(x_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=[1, 2], preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [10]:
len(v.get_feature_names())

253929

In [11]:
v_x_train = v.transform(x_train)

In [12]:
from sklearn.svm import SVC

clf = SVC(kernel='linear',C=1.0,probability=True)
clf.fit(v_x_train,y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [39]:
from sklearn.metrics import classification_report
y_pred = clf.predict(v.transform(x_test))
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    negative       0.81      0.80      0.81      2357
     neutral       0.33      0.42      0.37       626
    positive       0.94      0.92      0.93      7040

    accuracy                           0.86     10023
   macro avg       0.69      0.71      0.70     10023
weighted avg       0.87      0.86      0.86     10023



In [13]:
clf.score(v.transform(x_test),y_test)

0.8588247031826798

In [29]:
print(clf.predict(v.transform(['not good','bad','its fucking good','good','very bad','not bad','bad phone','ok ok phone'])),clf.predict_proba(v.transform(['not good','not bad','bad'])))

['negative' 'negative' 'positive' 'positive' 'negative' 'positive'
 'negative' 'positive'] [[0.64492965 0.11305268 0.24201767]
 [0.11982883 0.08780617 0.792365  ]
 [0.55689638 0.09664434 0.34645928]]


In [15]:
from sklearn import naive_bayes

In [16]:
clf_NB = naive_bayes.MultinomialNB()
clf_NB.fit(v_x_train,y_train)
clf_NB.score(v_x_train,y_train)


0.9221723326918965

In [17]:
clf_NB.score(v.transform(x_test),y_test)

0.8592237852938241

In [37]:
from sklearn.metrics import classification_report
y_pred = clf_NB.predict(v.transform(x_test))
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    negative       0.77      0.85      0.81      2110
     neutral       0.04      0.89      0.08        37
    positive       0.99      0.86      0.92      7876

    accuracy                           0.86     10023
   macro avg       0.60      0.87      0.60     10023
weighted avg       0.94      0.86      0.89     10023



In [18]:
clf_NB.predict(v.transform(['bad','good','very bad','not bad','not good','bad phone','ok ok phone']))

array(['positive', 'positive', 'negative', 'positive', 'positive',
       'negative', 'neutral'], dtype='<U8')

In [19]:
clf_NB2 = naive_bayes.BernoulliNB()

In [20]:
clf_NB2.fit(v_x_train, y_train)
clf_NB2.score(v_x_train,y_train)

0.7618131280735514

In [21]:
clf_NB2.score(v.transform(x_test),y_test)

0.7263294422827496

In [22]:
clf_NB2.predict(v.transform(['bad','good','very bad','not bad','not good','bad phone','ok ok phone']))

array(['positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive'], dtype='<U8')

* Tfidf

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=5).fit(x_train)
len(vect.get_feature_names())

5561

In [24]:
clf_NB2.fit(vect.transform(x_train),y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [25]:
clf_NB2.score(vect.transform(x_train),y_train)

0.7187940987812701

In [26]:
clf_NB2.score(vect.transform(x_test),y_test)

0.6997904818916492

In [27]:
clf_NB2.predict(vect.transform(['bad','good','very bad','not bad','not good','bad phone','ok ok phone']))

array(['positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive'], dtype='<U8')

In [41]:
from sklearn.metrics import classification_report
y_pred = clf_NB2.predict(vect.transform(x_test))
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    negative       0.46      0.72      0.56      1479
     neutral       0.26      0.16      0.20      1319
    positive       0.83      0.79      0.81      7225

    accuracy                           0.70     10023
   macro avg       0.52      0.56      0.52     10023
weighted avg       0.70      0.70      0.69     10023

